In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

## LangSmith

https://smith.langchain.com

In [2]:
# LANGCHAIN_TRACING_V2=true
# LANGCHAIN_ENDPOINT=https://api.smith.langchain.com
# LANGCHAIN_API_KEY=<your-api-key>
# LANGCHAIN_PROJECT=<your-project>  # if not specified, defaults to "default"

In [3]:
#!pip install langsmith

In [4]:
MODEL_GPT = 'gpt-4o-mini'

In [5]:
from langchain_openai import ChatOpenAI
from langchain.callbacks.tracers import LangChainTracer

# llm = ChatOpenAI()
llm = ChatOpenAI(model=MODEL_GPT)
tracer = LangChainTracer(project_name="Napoleon v1")
llm.predict("How many brothers had Napoleon Bonaparte?", callbacks=[tracer])

C:\Users\Pavel\AppData\Local\Temp\ipykernel_2700\4226358080.py:7: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm.predict("How many brothers had Napoleon Bonaparte?", callbacks=[tracer])


'Napoleon Bonaparte had three brothers: Joseph, Lucien, and Jérôme. He was the second of eight siblings, and his family had a significant influence on his life and career.'

### See updates in Projects Area in LangSmith

## Basic LangSmith Operations

### Create new project with LangChainTracer

In [6]:
from langchain.callbacks.tracers import LangChainTracer

tracer = LangChainTracer(project_name="Churchill v1")
llm.predict("How old was Churchill when he was appointed PM?", callbacks=[tracer])

'Winston Churchill was appointed Prime Minister of the United Kingdom on May 10, 1940. He was born on November 30, 1874, which made him 65 years old at the time of his appointment.'

### Check updates in Projects Area in LangSmith

### Alternative way to do the same

In [7]:
from langchain.callbacks import tracing_v2_enabled

with tracing_v2_enabled(project_name="Cyrus v1"):
    llm.predict("When did Cyrus The Great reign in Persia?")

## Creating Tags in LangSmith Projects

In [8]:
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# llm = ChatOpenAI(temperature=0, tags=["History"])
llm = ChatOpenAI(model=MODEL_GPT, temperature=0, tags=["History"])

prompt = PromptTemplate.from_template("Say {input}")

chain = LLMChain(
    llm=llm, 
    prompt=prompt, 
    tags=["Cyrus", "Persia"])

chain("When did the first Cyrus king reign in Persia?", tags=["Cyrus"])

C:\Users\Pavel\AppData\Local\Temp\ipykernel_2700\3592055948.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(
C:\Users\Pavel\AppData\Local\Temp\ipykernel_2700\3592055948.py:15: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain("When did the first Cyrus king reign in Persia?", tags=["Cyrus"])


{'input': 'When did the first Cyrus king reign in Persia?',
 'text': 'Cyrus the Great, the founder of the Achaemenid Empire in Persia, reigned from approximately 559 to 530 BCE. He is known for uniting the Medes and Persians and for his conquests that expanded the empire significantly. His reign marked the beginning of a new era in Persian history.'}

### See that this went to default project since we did not set that differently

## Creating Groups in LangSmith Projects

In [9]:
from langchain.callbacks.manager import (
    trace_as_chain_group
)

with trace_as_chain_group("American History v1") as group_manager:
    pass

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

# roman_llm = ChatOpenAI(temperature=0.9)
roman_llm = ChatOpenAI(model=MODEL_GPT, temperature=0.9)

prompt = PromptTemplate(
    input_variables=["question"],
    template="What is the answer to {question}?",
)

chain = LLMChain(
    llm=roman_llm, 
    prompt=prompt
)

with trace_as_chain_group("Roman History v1") as group_manager:
    chain.run(question="Who did Julius Caesar marry?", callbacks=group_manager)
    chain.run(question="Where did Julius Caesar fight?", callbacks=group_manager)
    chain.run(question="What was the name of the horse of Julius Caesar?", callbacks=group_manager)

C:\Users\Pavel\AppData\Local\Temp\ipykernel_2700\899673046.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run(question="Who did Julius Caesar marry?", callbacks=group_manager)


## LangSmith Client

In [10]:
from langsmith import Client

client = Client()
project_runs = client.list_runs(project_name="default")
project_runs

<generator object Client.list_runs at 0x0000021BA05A3880>

In [11]:
from datetime import datetime, timedelta

todays_runs = client.list_runs(
    project_name="default",
    start_time=datetime.now() - timedelta(days=1),
    run_type="llm",
)
todays_runs

<generator object Client.list_runs at 0x0000021BA05A3E20>

In [12]:
# for run in todays_runs:
#     print(run)

In [13]:
# todays_runs = client.list_runs(
#     project_name="Churchill v1",
#     start_time=datetime.now() - timedelta(days=1),
#     run_type="llm",
# )

# for run in todays_runs:
#     print(run)

## Adding metadata to filter runs

In [14]:
# chat_model = ChatOpenAI()
chat_model = ChatOpenAI(model=MODEL_GPT)
chain = LLMChain.from_string(
    llm=chat_model, 
    template="What's the answer to {input}?")

chain(
    {"input": "Who was the companion of Don Quixote?"}, 
    metadata={"source": "Cervantes"}
)

{'input': 'Who was the companion of Don Quixote?',
 'text': 'The companion of Don Quixote is Sancho Panza. He is a peasant who becomes Don Quixote\'s loyal squire and accompanies him on his adventures throughout Miguel de Cervantes\' novel "Don Quixote." Sancho provides a contrast to Don Quixote\'s idealism with his pragmatic and down-to-earth approach.'}

In [15]:
runs = list(client.list_runs(
    project_name="default",
    filter='has(metadata, \'{"source": "Cervantes"}\')',
))

#print(list(runs))

## Evaluating LLM App with Test Dataset in LangSmith

In [16]:
from langsmith import Client

example_inputs = [
  ("What is the largest mammal?", "The blue whale"),
  ("What do mammals and birds have in common?", "They are both warm-blooded"),
  ("What are reptiles known for?", "Having scales"),
  ("What's the main characteristic of amphibians?", "They live both in water and on land"),
]

client = Client()

dataset_name = "Elementary Animal Questions v1"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name, 
    description="Questions and answers about animal phylogenetics.",
)

for input_prompt, output_answer in example_inputs:
    client.create_example(
        inputs={"question": input_prompt},
        outputs={"answer": output_answer},
        dataset_id=dataset.id,
    )

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

In [17]:
from langsmith import Client
from langchain.smith import RunEvalConfig, run_on_dataset

evaluation_config = RunEvalConfig(
    evaluators=[
        "qa",
        "context_qa",
        "cot_qa",
    ]
)

In [18]:
client = Client()
# llm = ChatOpenAI()
llm = ChatOpenAI(model=MODEL_GPT)
run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=llm,
    client=client,
    evaluation=evaluation_config,
    project_name="evalproject v1",
)

LangSmithConflictError: Conflict for https://api.smith.langchain.com/sessions. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/sessions', '{"detail":"Session already exists."}')